In [1]:
from dl.simple_cnn import SimpleCNN
from dl.shuffle_net import ShuffleNetV2
from dl.mobile_net import MobileNET
from dl.wide_resnet import WideResNet
from dl.capsule_net import CapsuleNet
from dl.callbacks.plot_loss import PlotLosses
from data.data_set import get_data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, TerminateOnNaN, EarlyStopping, ModelCheckpoint, TensorBoard
from config import DL_MODELS_PATH, TB_LOGS_PATH
import os
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from swa.tfkeras import SWA
from dl.callbacks.cyclical_lr import CyclicLR
from plotly.offline import init_notebook_mode
from visualization.visualize_history import plot_accuracy_and_loss

init_notebook_mode(connected=True)

2019-12-18 20:57:14,031 : INFO : Checking/creating directories...
2019-12-18 20:57:14,033 : INFO : Directories are set.


In [2]:
((X_train, y_train), (X_val, y_val)) = get_data()

In [3]:
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range= .8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = .1, # Randomly zoom image 
        width_shift_range= .1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range= .1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=.05,  # randomly flip images horizontally
        vertical_flip=False)  # randomly flip images vertically

In [4]:
def get_callbacks(model_name, use_cyclic_lr=False, plot=False):
    callbacks = [TerminateOnNaN(), EarlyStopping(monitor='val_accuracy', patience=5)]
    callbacks.append(ModelCheckpoint(os.path.join(DL_MODELS_PATH, model_name + '-{epoch:02d}-{val_accuracy:.2f}.hdf5'), monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max'))
    tb_path = os.path.join(TB_LOGS_PATH, model_name)
    pathlib.Path(tb_path).mkdir(parents=True, exist_ok=True)
    callbacks.append(TensorBoard(log_dir=tb_path))
    callbacks.append(SWA(start_epoch=5, lr_schedule='constant', swa_lr=0.001, verbose=1))
    if plot:
        callbacks.append(PlotLosses())
    if use_cyclic_lr:
        callbacks.append(CyclicLR(base_lr=0.001, max_lr=0.006, step_size=2000.))
    else:
        rlronp =  ReduceLROnPlateau(monitor='val_loss',
                                    patience=3,
                                    verbose=1,
                                    factor=0.5,
                                    min_lr=0.00005)
        
        callbacks.append(rlronp)
    return callbacks

In [5]:
epochs = 100
Learning_rate = 0.001
decay= 5 * Learning_rate / epochs
optimizers = {'adam': Adam(lr=Learning_rate, decay= 3 * Learning_rate / epochs),
              'rmsprop': RMSprop(lr=Learning_rate, rho=0.9, epsilon=1e-08, decay= 0)}

In [ ]:
epochs = 50
histories = {}
models = {}

for opt_name, opt in optimizers.items():
    '''
    # Train capsule net
    model_name = f'capsnet_{opt_name}'
    model = WideResNet(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                          16, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model
    '''
    
    # Train wide resnet net
    model_name = f'wide_resnet_{opt_name}'
    model = WideResNet(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                              32, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model

    # Train mobile_net
    model_name = f'mobile_net_{opt_name}'
    model = MobileNET(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                            16, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model
        
    # Train a basic cnn
    model_name = f'simple_cnn_{opt_name}'
    model = SimpleCNN(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                            2048, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model
    
    # Train shufflenetv2
    model_name = f'shuffle_netv2_{opt_name}'
    model = ShuffleNetV2(opt)
    model = ShuffleNetV2(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                              256, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model

2019-12-18 20:57:17,643 : WARNING : Tensor._shape is private, use Tensor.shape instead. Tensor._shape will eventually be removed.


2019-12-18 20:57:17,729 : WARNING : Tensor._shape is private, use Tensor.shape instead. Tensor._shape will eventually be removed.
Resizing training images: 60000it [00:06, 9842.57it/s] 
Resizing training images: 10000it [00:00, 13009.10it/s]


Model uses batch normalization. SWA will require last epoch to be a forward pass and will run with no learning rate
Epoch 1/50
1874/1875 [============================>.] - ETA: 0s - loss: 0.6044 - accuracy: 0.7799 - categorical_crossentropy: 0.6044 - categorical_accuracy: 0.7799
Epoch 00001: val_categorical_accuracy improved from -inf to 0.78520, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/wide_resnet_adam-01-0.79.hdf5
1875/1875 [==============================] - 428s 228ms/step - loss: 0.6043 - accuracy: 0.7800 - categorical_crossentropy: 0.6043 - categorical_accuracy: 0.7800 - val_loss: 0.5805 - val_accuracy: 0.7852 - val_categorical_crossentropy: 0.5807 - val_categorical_accuracy: 0.7852
Epoch 2/50
1874/1875 [============================>.] - ETA: 0s - loss: 0.3848 - accuracy: 0.8606 - categorical_crossentropy: 0.3848 - categorical_accuracy: 0.8606
Epoch 00002: val_categorical_accuracy improved from 0.78520 to 0.85000, saving model to /home/cenk/Research/

Epoch 15/50
1874/1875 [============================>.] - ETA: 0s - loss: 0.1607 - accuracy: 0.9419 - categorical_crossentropy: 0.1607 - categorical_accuracy: 0.9419
Epoch 00015: val_categorical_accuracy improved from 0.93840 to 0.93870, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/wide_resnet_adam-15-0.94.hdf5
1875/1875 [==============================] - 421s 224ms/step - loss: 0.1607 - accuracy: 0.9419 - categorical_crossentropy: 0.1607 - categorical_accuracy: 0.9419 - val_loss: 0.1774 - val_accuracy: 0.9387 - val_categorical_crossentropy: 0.1774 - val_categorical_accuracy: 0.9387
Epoch 16/50
1874/1875 [============================>.] - ETA: 0s - loss: 0.1547 - accuracy: 0.9437 - categorical_crossentropy: 0.1547 - categorical_accuracy: 0.9437
Epoch 00016: val_categorical_accuracy did not improve from 0.93870
1875/1875 [==============================] - 423s 226ms/step - loss: 0.1547 - accuracy: 0.9437 - categorical_crossentropy: 0.1547 - categorical_accuracy

Resizing training images: 60000it [00:05, 10836.66it/s]
Resizing training images: 10000it [00:00, 13203.54it/s]


Epoch 1/50
3749/3750 [============================>.] - ETA: 0s - loss: 0.7528 - accuracy: 0.7252 - categorical_crossentropy: 0.7528 - categorical_accuracy: 0.7252
Epoch 00001: val_categorical_accuracy improved from -inf to 0.10000, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/mobile_net_adam-01-0.10.hdf5
3750/3750 [==============================] - 604s 161ms/step - loss: 0.7528 - accuracy: 0.7252 - categorical_crossentropy: 0.7528 - categorical_accuracy: 0.7252 - val_loss: 2.7076 - val_accuracy: 0.1000 - val_categorical_crossentropy: 2.7076 - val_categorical_accuracy: 0.1000
Epoch 2/50
3749/3750 [============================>.] - ETA: 0s - loss: 0.4847 - accuracy: 0.8241 - categorical_crossentropy: 0.4847 - categorical_accuracy: 0.8241
Epoch 00002: val_categorical_accuracy improved from 0.10000 to 0.83700, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/mobile_net_adam-02-0.84.hdf5
3750/3750 [==============================] - 604s 16

3750/3750 [==============================] - 601s 160ms/step - loss: 0.2316 - accuracy: 0.9152 - categorical_crossentropy: 0.2316 - categorical_accuracy: 0.9152 - val_loss: 0.2326 - val_accuracy: 0.9132 - val_categorical_crossentropy: 0.2326 - val_categorical_accuracy: 0.9132
Epoch 16/50
3749/3750 [============================>.] - ETA: 0s - loss: 0.2278 - accuracy: 0.9157 - categorical_crossentropy: 0.2278 - categorical_accuracy: 0.9157
Epoch 00016: val_categorical_accuracy did not improve from 0.91320
3750/3750 [==============================] - 599s 160ms/step - loss: 0.2278 - accuracy: 0.9157 - categorical_crossentropy: 0.2278 - categorical_accuracy: 0.9157 - val_loss: 0.2614 - val_accuracy: 0.9039 - val_categorical_crossentropy: 0.2614 - val_categorical_accuracy: 0.9039
Epoch 17/50
3749/3750 [============================>.] - ETA: 0s - loss: 0.2215 - accuracy: 0.9183 - categorical_crossentropy: 0.2215 - categorical_accuracy: 0.9183
Epoch 00017: val_categorical_accuracy improved fr

In [ ]:
plot_accuracy_and_loss(histories)